In [ ]:
from easyocr import Reader
import argparse
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
def cleanup_text(text):
	# strip out non-ASCII text so we can draw the text on the image
	# using OpenCV
	return "".join([c if ord(c) < 128 else "" for c in text]).strip()

In [ ]:
image_path = r"Pictures\Stamp.PNG"
image_path2 = r"Pictures\Stamp2.PNG"
image_path3 = r"Pictures\Stamp3.PNG"

image = cv2.imread(r"Pictures\GreyOutput.PNG")


In [ ]:
reader = Reader(["en"], False)
results = reader.readtext(image, allowlist ='0123456789')
results



In [ ]:
import os
import re
from PIL import Image
from easyocr import Reader
import argparse
import cv2
import os
import pdfHelper
import DateConversion
import PIL
import numpy as np
import scipy.ndimage as nd
import pymupdf

temp_png_top = r"Documents\NewWorkflow\Temporary\topTemp.png"
temp_png_bottom = r"Documents\NewWorkflow\Temporary\bottomTemp.png"

image = cv2.imread(temp_png_bottom)

def clean_text(a):
    return re.sub('[^0-9]','', a)

stamp_regex = r"2[0-9]{6}[0-9]$"
stamp_regex2 = r"(19|20)\d{2}(0[1-9]|1[0,1,2])(0[1-9]|[12][0-9]|3[01])"
compiled_regex = re.compile(stamp_regex)
compiled_regex2 = re.compile(stamp_regex2)
print(re.search(compiled_regex2, "20151230")[1])

reader = Reader(["en"], False)

def solo_test(path):
    combined = path
    file = os.path.basename(path)
    correct = file.split("_")[3]
    # pdfHelper.pdfpage_to_image(combined, temp_png_top, 0)
    # array = pdfHelper.image_to_array(temp_png_top, 0, .1)
    doc = pymupdf.open(combined)
    # Loop over pages and render
    page = doc.load_page(0)
    pix = page.get_pixmap(dpi=300)  # render page to an image
    doc.close()

    image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    array = np.asarray(image, np.int32)

    #array = cv2.imread(r"Pictures\RedHelp.png")
    #array = cv2.cvtColor(array, cv2.COLOR_BGR2RGB)
    
    array = nd.maximum_filter(array, 3)

    array[array < 90] = 255
    array[array < 200] = 0
    array = nd.median_filter(array, 3)
    array[array < 220] = 0
    array = nd.maximum_filter(array, 3)
    array[array > 1] = 255
    array = nd.gaussian_filter(array, 3)
    array = nd.maximum_filter(array, 3)
    array[array < 200] = 0
    array[array > 1] = 255
    #array = nd.maximum_filter(array, 3)
    
    #array = array.astype(np.int32)
    pdfHelper.save_image_array(temp_png_top, array)
    image = cv2.imread(temp_png_top)
    results = reader.readtext(image, allowlist ='0123456789')
    found = None
    for r in results:
        result_name = clean_text(r[1])
        extract = re.search(DateConversion.regex3, result_name)
        if(extract is not None):
            found = extract.group()
            break
    if found is None:
        print("  neg" + "  Correct: " + str(correct))
    elif found == correct:
        print("  pos" + "  Correct: " + str(found))
    else :
        print("  false pos" + "  Correct: " + str(correct) + "  Incorrect: " + str(found))
    return results, array

results, array = solo_test(r"Documents\NewWorkflow\Final\DCR_15523_RE_20180719_Renewal-DS.pdf")
print(results)



In [ ]:
stamp_regex = r"2[0-9]{6}[0-9]$"
stamp_regex2 = r"(19|20)\d{2}(0[1-9]|1[0,1,2])(0[1-9]|[12][0-9]|3[01])"
compiled_regex = re.compile(stamp_regex)
compiled_regex2 = re.compile(stamp_regex2)
r = re.search(compiled_regex2, "20151230")
print(r[0])



In [ ]:
import imageio.v2 as imageio

fig, ax = plt.subplots()
ax.imshow(imageio.imread(temp_png_top), interpolation=None)

# loop over the results
for (bbox, text, prob) in results:
	# display the OCR'd text and associated probability
	print("[INFO] {:.4f}: {}".format(prob, text))
	# unpack the bounding box
	(tl, tr, br, bl) = bbox
	tl = (int(tl[0]), int(tl[1]))
	tr = (int(tr[0]), int(tr[1]))
	br = (int(br[0]), int(br[1]))
	bl = (int(bl[0]), int(bl[1]))
	# cleanup the text and draw the box surrounding the text along
	# with the OCR'd text itself
	text = cleanup_text(text)

	font = {'family' : 'normal',
        	'size'   : 5}
	rect = patches.Rectangle(bl, tr[0] - bl[0], tr[1] - bl[1], linewidth=1, edgecolor='r', facecolor='none')
	plt.text(tl[0], tl[1], text, **font)

	ax.add_patch(rect)
	# cv2.rectangle(image, tl, br, (0, 255, 0), 2)
	# cv2.putText(image, text, (tl[0], tl[1] - 10),
	# 	cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
# show the output image
